# Landslide Hazard Analysis Using SAR
Intro to the use case/methods used

Search for SLC images over Haiti in a defined time period

In [ ]:
import os
from datetime import datetime

# start_time = datetime.strptime('2021-05-01T23:59', '%Y-%m-%dT%H:%M')
start_time = datetime.strptime('2020-08-01T23:59', '%Y-%m-%dT%H:%M')
end_time = datetime.strptime('2021-08-16T23:59', '%Y-%m-%dT%H:%M')
event_time = datetime.strptime('2021-08-14T00:00', '%Y-%m-%dT%H:%M')

import asf_search
scenes_to_submit = []
wkt = 'POLYGON((-74.6 18.0,-73.0 18.0,-73.0 18.8,-74.6 18.8, -74.6 18.0))'
results_descending = asf_search.geo_search(platform=[asf_search.PLATFORM.SENTINEL1], intersectsWith=wkt, processingLevel='SLC', start=start_time, end=end_time, relativeOrbit=142, frame=530)
# [scenes_to_submit.append(result.properties['sceneName']) for result in results]
# print(f'There are {len(scenes_to_submit)} scenes in the AOI between {start_time} and {end_time}')

Request HyP3 processing of Haiti data for two years

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

properties = [result.geojson()['properties'] for result in results_descending]
geometries = [shape(result.geojson()['geometry']) for result in results_descending]
gdf = gpd.GeoDataFrame(properties, geometry=geometries,crs='EPSG:4326')
scenes_to_submit = list(gdf['sceneName'])
print(gdf.shape[0])

In [ ]:
gdf.explore(style_kwds=dict(color='black', fill=False))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plot_gdf = gdf[['sceneName', 'startTime']].copy()
plot_gdf['startTime'] = pd.to_datetime(plot_gdf['startTime'])
plot_gdf = plot_gdf.sort_values('startTime').reset_index()

f, ax = plt.subplots(1,1,figsize=(7,7))
ax.scatter(pd.to_datetime(plot_gdf['startTime']), plot_gdf.index, s=5)
ax.set(xlabel='Date', ylabel='Image Number')
ax.axvline(event_time, color='red')

In [ ]:
import hyp3_sdk as sdk

print(f'Submitting {len(scenes_to_submit)} jobs')
hyp3 = sdk.HyP3()
rtc_jobs = sdk.Batch()
for scene in scenes_to_submit:
    rtc_jobs += hyp3.submit_rtc_job(granule=scene, name='IGARSS-HAITI')

In [ ]:
data_dir_path = 'haiti_rtcs'
rtc_jobs = hyp3.watch(rtc_jobs)
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
file_list = succeeded_jobs.download_files(location=data_dir_path)

In [ ]:
import hyp3_sdk as sdk
from pathlib import Path

files = Path('haiti_rtcs').glob('*')
for file in files:
    sdk.util.extract_zipped_product(file, delete=False)

Crop all images to same extent (e.g. InSAR Notebook)

In [ ]:
from pathlib import Path
from typing import List, Union
from osgeo import gdal


def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

In [ ]:
from pathlib import Path
from typing import List, Union

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_VV.tif', '_VH.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [ ]:
data_dir = Path('haiti_rtcs')
files = list(data_dir.glob('**/*_VV.tif'))
overlap = get_common_overlap(files)
clip_hyp3_products_to_common_overlap(data_dir, overlap)

Load the geotiffs into Xarray with datetime stamps for each image

In [6]:
from datetime import datetime
from pathlib import Path
import rioxarray as rxr
import xarray as xr
rtc_files = list(Path('haiti_rtcs').glob('*/*_VV_clipped.tif'))
xds = xr.open_dataset(rtc_files[0], engine='rasterio')
time = datetime.strptime(rtc_files[0].stem[7:22], '%Y%m%dT%H%M%S')
xds = xds.assign_coords(time=time)
xds

<xarray.Dataset>
Dimensions:      (band: 1, x: 9422, y: 7375)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 5.057e+05 5.057e+05 ... 7.883e+05 7.883e+05
  * y            (y) float64 2.179e+06 2.179e+06 ... 1.958e+06 1.958e+06
    spatial_ref  int64 ...
    time         datetime64[ns] 2021-07-22T10:47:35
Data variables:
    band_data    (band, y, x) float32 ...

In [9]:
import os
import xarray as xr
from datetime import datetime
from pathlib import Path

def preprocess(da_orig, file_type: str='vv'):
    '''function that should return an xarray object with time dimension and associated metadata given a path to a single RTC scene, if its dualpol will have multiple bands, currently just as 2 data arrays but could merge.
    goal would be to apply this a list of directories for different RTC products, return cube along time dimension - I think?
    - for concatenating, would need to check footprints and only take products with the same footprint, or subset them all to a common AOI? '''
    da = da_orig.copy()
    da = da.rename({'band_data': file_type}).squeeze()
    fname = os.path.basename(da_orig['band_data'].encoding['source'])
    time = datetime.strptime(fname[7:22], '%Y%m%dT%H%M%S')
    da = da.assign_coords({'time': time})
    da = da.expand_dims('time')
    da = da.drop_duplicates(dim=['x', 'y'])

    return da

fpaths_vv = list(Path('haiti_rtcs').glob('*/*_VV_clipped.tif'))
rtc_timeseries = xr.open_mfdataset(paths = fpaths_vv, preprocess = preprocess, chunks = 'auto', engine='rasterio', data_vars='minimal', coords='minimal', concat_dim='time', combine='nested', parallel=True)

In [10]:
rtc_timeseries

<xarray.Dataset>
Dimensions:      (x: 9422, y: 7375, time: 32)
Coordinates:
    band         int64 1
  * x            (x) float64 5.057e+05 5.057e+05 ... 7.883e+05 7.883e+05
  * y            (y) float64 2.179e+06 2.179e+06 ... 1.958e+06 1.958e+06
    spatial_ref  int64 0
  * time         (time) datetime64[ns] 2021-07-22T10:47:35 ... 2021-03-24T10:...
Data variables:
    vv           (time, y, x) float32 dask.array<chunksize=(1, 5124, 6547), meta=np.ndarray>

Pre-temporal averaging, create time series plot of single pixel (or group of pixels)

Create a temporal average for both pre and post EQ

In [ ]:
import numpy as np

date_bins = [start_time, event_time, end_time]
date_bin_labels = ["preevent", "postevent"]

vv = vv.groupby_bins("acq_date", date_bins, labels=date_bin_labels)
mean_vv = vv.mean(dim='acq_date')

Perform a log difference of the two resulting images.

In [ ]:
log_diff = 10*np.log10(mean_vv['vv'][0]/mean_vv['vv'][1])

import matplotlib.pyplot as plt

plt.matshow(log_diff)
plt.colorbar()
plt.show()

Discuss options for thresholding the difference image

Use the COP30 DEM to remove change detections in areas with slopes <5 degrees use gdaldem

In [ ]:
from dem_stitcher import stitch_dem


dem_file = data_dir_path / 'DEM.tif'
bounds = [-74.6, 18.0, -73.0, 18.8]
X, p = stitch_dem(bounds,
                  dem_name='glo_30',  # Global Copernicus 30 meter resolution DEM
                  dst_ellipsoidal_height=False,
                  dst_area_or_point='Point')

import rasterio

with rasterio.open(dem_file, 'w', **p) as ds:
   ds.write(X, 1)
   ds.update_tags(AREA_OR_POINT='Point')

process_dem_file = data_dir_path / 'DEM_processed.tif'
gdal.DEMProcessing(destName=str(process_dem_file), srcDS=str(dem_file), processing="slope", format="Gtiff", slopeFormat="degree")

height, width = np.shape(log_diff)
gdal.Warp(str(process_dem_file), str(process_dem_file), dstSRS='EPSG:4326',
                  outputBounds=bounds, width=width, height=height, resampleAlg='nearest', format='GTiff')

dem = gdal.Open(str(process_dem_file), gdal.GA_ReadOnly).ReadAsArray()
dem_mask = dem > 5

log_diff = log_diff.where(dem_mask)

 Plot a histogram for the difference image (might have to subset histogram source to area with equal amounts of landslides and non-landslides).

In [ ]:
import matplotlib.pyplot as plt

plt.hist(log_diff)
plt.show()

Create a figure to interactively view the data and set a threshold using matplotlib

In [ ]:
plt.matshow(log_diff.where(dem_mask))
plt.show()